In [37]:
import praw
import pandas as pd
from datetime import datetime

# Establish connection with Reddit API
reddit = praw.Reddit(user_agent='Comment Extraction (by /u/finaltutorial)',
                     client_id='GHbnn7wSkzKftg',client_secret="PElco2geJIcllSCAjhl2DYMQ1aODqQ",
                     username='finaltutorial', password='320finaltutorial')

In [38]:
# Perform data extraction on the /r/finance subreddit
comm_list = []
header_list = []
subs = []

# Sample 10 hottests posts on the finance subreddit
hot_posts = reddit.subreddit('finance').hot(limit=10)

for sub in hot_posts:
    
    # Store general data from hot posts
    subs.append([sub.title, sub.score, sub.id, sub.subreddit, sub.url, sub.num_comments, sub.selftext, datetime.fromtimestamp(sub.created)])

    # Generate list of comment IDs within posts
    sub.comments.replace_more(limit=None)
    comment_queue = sub.comments[:]  # Seed with top-level
    
    # Compile list of nested replies
    while comment_queue:
        header_list.append(sub.title)
        comment = comment_queue.pop(0)
        comm_list.append(comment.body)
        t = []
        t.extend(comment.replies)
        while t:
            header_list.append(sub.title)
            reply = t.pop(0)
            comm_list.append(reply.body)
            
# Generate general dataframe
subs = pd.DataFrame(subs,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
subs.head()

,title,score,id,subreddit,url,num_comments,body,created
0,"Moronic Monday, December 14, 2020 - Your Weekl...",8,kcx8ya,finance,https://www.reddit.com/r/finance/comments/kcx8...,44,This is your safe place for questions on finan...,2020-12-14 21:04:16
1,I built a website to easily analyze the portfo...,155,kegs4g,finance,https://cheaperthanguru.com/,17,,2020-12-17 04:07:38
2,Massachusetts Regulators to File Complaint Aga...,391,keagtt,finance,https://www.wsj.com/articles/massachusetts-reg...,101,,2020-12-16 22:45:51
3,High-Frequency Traders Push Closer to Light Sp...,658,kdn0b9,finance,https://www.wsj.com/articles/high-frequency-tr...,145,,2020-12-15 22:47:46
4,Domino's Pizza Analysis (NYSE: DPZ),0,keht0d,finance,https://weekly10k.substack.com/p/dominos?r=abc...,0,,2020-12-17 04:58:18


In [32]:
# Assemble comments from top posts in dataframe
df = pd.DataFrame(header_list)
df['comm_list'] = comm_list

df.head()

,0,comm_list
0,"Moronic Monday, December 14, 2020 - Your Weekl...",Hypothetical that I've never been able to get ...
1,"Moronic Monday, December 14, 2020 - Your Weekl...","Yes, because the potential for share buybacks ..."
2,"Moronic Monday, December 14, 2020 - Your Weekl...",Depends on what you mean by “value” - one defi...
3,"Moronic Monday, December 14, 2020 - Your Weekl...",As a shareholder you own a percentage of the c...
4,"Moronic Monday, December 14, 2020 - Your Weekl...","Depends on the company, but if we assume in th..."


In [33]:
# Perform pre-processessing and store in csv
df.columns = ['title','comments']
df['comments'] = df['comments'].apply(lambda x : x.replace('\n',''))
df.to_csv('finance_comments.csv',index = False)

df.head()

,title,comments
0,"Moronic Monday, December 14, 2020 - Your Weekl...",Hypothetical that I've never been able to get ...
1,"Moronic Monday, December 14, 2020 - Your Weekl...","Yes, because the potential for share buybacks ..."
2,"Moronic Monday, December 14, 2020 - Your Weekl...",Depends on what you mean by “value” - one defi...
3,"Moronic Monday, December 14, 2020 - Your Weekl...",As a shareholder you own a percentage of the c...
4,"Moronic Monday, December 14, 2020 - Your Weekl...","Depends on the company, but if we assume in th..."
